# Модуль 18. Практическая работа

## Цели работы

- Попрактиковаться в предобработке текстовых данных (токенизация, лемматизация, векторизация).

- Построить две модели классификации с разными способами векторизации текстов (`CountVectorizer` и `TfIdfVectorizer`).
- Сравнить качество моделей.

## Что нужно сделать

1. Загрузите датасет из файла, приложенного к заданию (`m18_jokes_dataset.csv`).

1. Закодируйте целевую переменную с помощью `LabelEncoder`.
1. Разделите выборку на тренировочную и тестовую.
1. Используя знания из модуля, подготовьте функцию `preprocess`, которая:
- удаляет стоп-слова в соответствии со списком `nltk.corpus.stopwords`;
- приводит слова к нормальной форме с помощью `pymorphy2`.
5. Создайте экзепляры векторизаторов `CountVectorizer` и `TfIdfVectorizer`. В качестве функции предобработки данных оба должны использовать функцию `preprocess`.

1. Векторизуйте признаки тренировочной и тестовой выборок. Сохраните результаты `CountVectorizer` в `X_train_count` и `X_test_count`, а результаты `TfIdfVectorizer` – в `X_train_tfidf` и `X_test_tfidf`.
1. Обучите две модели классификации, например `LinearSVC`.
1. Оцените работу обеих моделей на тестовой выборке, используя `f1_score` с методом усреднения `macro` (параметр `average='macro'`) – это арифметическое среднее f1-метрик, рассчитанных для каждого класса. 

## Что оценивается

- Все шаги выполнены правильно.

- Код решения соответствует референсному коду, который есть у куратора. 
- Обучены две модели, которые используют текстовые данные, обработанные разными векторизаторами (`CountVectorizer` и `TfIdfVectorizer`). Для обеих моделей вычислен усреднённый `f1_score` (метод усреднения — `macro`). 

## Информация о задаче

### Описание датасета

`m18_jokes_dataset.csv` – датасет, подготовленный на основе датасета, размещённого [на kaggle](https://www.kaggle.com/datasets/konstantinalbul/russian-jokes). Наш датасет содержит две колонки:

- **text** — текст анекдота (скорее всего, несмешного, но ваша задача не смеяться, а решить задачу классификации);

- **theme** — категория, к которой относится шутка. 

Датасет содержит данные для задачи многоклассовой классификации. В отличие от оригинального датасета, в нашем классов меньше, зато они идеально сбалансированы: в каждом ровно тысяча примеров. Так вам будет проще сконцентрироваться на текстовых данных.

### Порядок выполнения 

Напишите весь необходимый код в ячейках ниже.

### Рекомендации

- Обратите внимание, что модели sklearn (например, `LinearSVC`) умеют работать с разреженными матрицами (`scipy.sparse._csr.csr_matrix`), которые возвращают методы и `fit_transform` и `transform` векторизаторов `CountVectorizer` и `TfIdfVectorizer`.

- Когда подготавливаете векторизатор, вызывайте `fit_transform` на тренировочной выборке, а потом `fit` на тестовой, примерно так:
    > X_train_count = count_vectorizer.fit_transform(X_train)
    >
    > X_test_count = count_vectorizer.transform(X_test)

Затем используйте полученные результаты для обучения и контроля моделей `sklearn`.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("m18_jokes_dataset.csv")

df.head()

,theme,text
0,aforizmi,"Почему удар в спину наносят те, кого,как прави..."
1,aforizmi,Музы - пожалуй самые мудрые представительницы...
2,aforizmi,"Тяжело бухать всю ночь, особенно если ночь пол..."
3,aforizmi,ПОКУШЕНИЕ. НАЛИЧНОСТЬ.\r\n\r\n\r\n
4,aforizmi,"Когда медленно танцуешь, ничего не мешает...\r..."


In [3]:
# Датасет идеально сбалансирован
df.groupby("theme").size()

theme
aforizmi                 1000
meditsinskie             1000
narodnie                 1000
poshlie-i-intimnie       1000
pro-alkogolikov          1000
pro-armiu                1000
pro-detey                1000
pro-evreev               1000
pro-militsiyu            1000
pro-mugchin              1000
pro-novih-russkih        1000
pro-semyu                1000
pro-studentov            1000
pro-vovochku             1000
raznie                   1000
shkolnie-i-pro-shkolu    1000
tsitati                  1000
dtype: int64

In [4]:
label_encoder = LabelEncoder()
df["theme"] = label_encoder.fit_transform(df["theme"])
df.sample(5)

,theme,text
16,0,"Если хочешь больше, чем можешь - не имеешь и т..."
15650,15,Одна тетка как-то решила завести попугая. Пошл...
4262,4,"На фразу ""Да ты же будешь в зюзю пьяный!"" над..."
13550,13,"Вовочка спрашивает у мамы:\r\n- Мам, что за ры..."
8991,8,"(Litz): недавно понял, что помимо материнской..."


In [5]:
X, y = df["text"], df["theme"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = RegexpTokenizer("\w+")
russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()


def preprocess(text):
    stemmed_words = []
    for word in tokenizer.tokenize(text):
        word = word.lower()
        if word not in russian_stopwords:
            stemmed_words.append(morph.parse(word)[0].normal_form)
    return " ".join(stemmed_words)

In [7]:
count_vectorizer = CountVectorizer(preprocessor=preprocess)

tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess)

In [8]:
%%time

X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

CPU times: total: 54 s
Wall time: 50.7 s


In [9]:
%%time

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

CPU times: total: 54.7 s
Wall time: 51.8 s


In [10]:
svc_count = LinearSVC(max_iter=10000)

svc_count.fit(X_train_count, y_train)

y_pred_count = svc_count.predict(X_test_count)

print(classification_report(y_test, y_pred_count))

              precision    recall  f1-score   support

           0       0.37      0.61      0.46       199
           1       0.56      0.54      0.55       210
           2       0.51      0.43      0.47       200
           3       0.40      0.42      0.41       200
           4       0.90      0.89      0.90       212
           5       0.62      0.65      0.64       197
           6       0.95      0.94      0.94       204
           7       0.83      0.79      0.81       185
           8       0.89      0.72      0.80       206
           9       0.27      0.28      0.27       195
          10       0.82      0.85      0.83       213
          11       0.44      0.50      0.47       202
          12       0.72      0.75      0.73       201
          13       0.91      0.86      0.88       209
          14       0.14      0.12      0.13       207
          15       0.95      0.89      0.92       179
          16       0.56      0.36      0.44       181

    accuracy              

In [11]:
f1_score(y_test, y_pred_count, average='macro')

0.6266140496771212

In [12]:
%%time

svc_tfidf = LinearSVC(max_iter=10000)

svc_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = svc_tfidf.predict(X_test_tfidf)

print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.38      0.44      0.41       199
           1       0.64      0.66      0.65       210
           2       0.54      0.51      0.52       200
           3       0.42      0.53      0.47       200
           4       0.89      0.91      0.90       212
           5       0.71      0.75      0.73       197
           6       0.92      0.95      0.93       204
           7       0.82      0.84      0.83       185
           8       0.87      0.76      0.81       206
           9       0.36      0.30      0.33       195
          10       0.78      0.90      0.84       213
          11       0.46      0.49      0.47       202
          12       0.76      0.79      0.77       201
          13       0.88      0.87      0.88       209
          14       0.17      0.12      0.14       207
          15       0.96      0.87      0.91       179
          16       0.54      0.51      0.53       181

    accuracy              

In [13]:
f1_score(y_test, y_pred_tfidf, average='macro')

0.6533412680920231